In [2]:
import openpyxl
import numpy as np
import pandas as pd
from tqdm import tqdm
from gekko import GEKKO
import matplotlib.pyplot as plt 
from scipy.optimize import fsolve
from scipy.interpolate import UnivariateSpline
import scipy.integrate
import matplotlib.animation as animation
from sklearn.metrics import r2_score 

In [3]:
#Test  Cavenati
#Parameters for the simulation 
pressure                              = 3.2                                      # pressure during adsorption step, bar
pressure_init                         = 0.1
seg                                   = 100
T                                     = 303                                   #Solid initial Temperature, K
V_tank                                = 6e-5

ep                                    = 0.33                                     #Particle porosity 
ei                                    = 0.45                                     #External porosity
et                                    = ep+ei                                     #Total porosity
L                                     = 0.83                                    #Length of the bed, m
D                                     = 0.021                                #Diameter of the bed, m
rho                                   = 715                                     #Apparent(bulk) density, kg/m3
R                                     = 8.314                                   #Universal gas constant, J/(mol K)
Flow_rate                             = 1.5*6.72701607326312E-07
Flow_rate_purge                       = 0.6*6.72701607326312E-07
adsorption_time                       = 100                                     #s

y1_feed_pressurization                = 1                                    #Methane molar fraction     
y2_feed_pressurization                = 0                                   #Ethane molar fraction
y3_feed_pressurization                = 0                                   #Propane molar fraction
y4_feed_pressurization                = 0                                     #Butane molar fraction
y5_feed_pressurization                = 0                                   #CO2 molar fraction
y6_feed_pressurization                = 1 - y1_feed_pressurization  - y2_feed_pressurization - y3_feed_pressurization - y4_feed_pressurization  - y5_feed_pressurization                                 #Nitrogen molar fraction

                                                                                #Feed Parameters 
y1_feed_adsorption                    = y1_feed_pressurization                                    #Methane molar fraction     
y2_feed_adsorption                    = y2_feed_pressurization                                  #Ethane molar fraction
y3_feed_adsorption                    = y3_feed_pressurization                                    #Propane molar fraction
y4_feed_adsorption                    = y4_feed_pressurization                                     #Butane molar fraction
y5_feed_adsorption                    = y5_feed_pressurization                                   #CO2 molar fraction
y6_feed_adsorption                    = y6_feed_pressurization                                   #Nitrogen molar fraction

y1_feed_purge                         = 0
y2_feed_purge                         = 0
y3_feed_purge                         = 0
y4_feed_purge                         = 0
y5_feed_purge                         = 0
y6_feed_purge                         = 1

Pressure_purge                        = 0.1                                     #Bar

number_cycles = 46                                                                               ### Isotherm parameters ###
# Isotherm parameters
IP1_CH4 = 1.079e-6 #kmol/kg
IP2_CH4 = 2208 #1/bar
IP3_CH4 = 0.916 # 1/bar
IP4_CH4 = -130.9 # K

  
IP1_C2H6 = 0.0027 #kmol/kg
IP2_C2H6 = 0.0 #1/bar 
IP3_C2H6 = 2.66E-04 # 1/bar
IP4_C2H6 = 2833.77 # K

IP1_C3H8 = 0.0062 #kmol/kg
IP2_C3H8 = 0.0 #1/bar 
IP3_C3H8 = 3.75E-04 # 1/bar
IP4_C3H8 = 2795.28 # K

IP1_C4H10 = 0.0076 #kmol/kg
IP2_C4H10 = 0.0 #1/bar 
IP3_C4H10 = 0.0015 # 1/bar
IP4_C4H10 = 2600 # K

IP1_CO2 = 1.127e-5 #kmol/kg
IP2_CO2 = 1880 #(kmol/kg)/bar 
IP3_CO2 = 0.145 # 1/bar
IP4_CO2 = 777.9 # K

IP1_N2 = 0 #kmol/kg
IP2_N2 = 0 #(kmol/kg)/bar 
IP3_N2 = 0 # 1/bar
IP4_N2 = 0 # K

V_tank1                             = 1e-3                                                                               #Mass transfer coefficients, 1/s
MTC                                   = [0,0,0,0,0,0]                     #CH4,C2H6,C3H8,C4H10,CO2,N2

T_initial = 303
r_p = 5e-3
mu_g = 1.185e-2*0.001
k_gas = 2.198e-2
cp_gas = 34000 #J/kmol K
cp_solid = 878 #J/kgK
k_solid = 16 # W/mK
dH_ch4 = 16*10**6
dH_c2h6 = 25*10**6
dH_c3h8 = 28*10**6
dH_c4h10 = 30*10**6
dH_n2 = 10*10**6
dH_co2 = 16*10**6

exp = 2.7182

#Pump char
Qmax_pump = 0.000149805471647187
b_pump = 768250.064952021
n_pump = 0.970650968902519
k_pump = 3.2633941869566 

B_mu = 0.001
K = 0.058

In [4]:
for cycle_count in range(0,number_cycles):
    print(f"Staarted cycle {cycle_count+1}")
    MTC = [5.7E-5,0.025]
    m = GEKKO(remote = False)  # create GEKKO model
    tf = 70
    nt = int(tf/1) + 1
    m.time = np.linspace(0,tf,nt)
    if cycle_count == 0:
        y1_feed = 0.55
        y2_feed = 0.45
        y3_feed = 0
        L_seg = L/seg
        Area = np.pi * D**2/4
        P = [m.Var(pressure_init) for i in range(seg)] 
        y1 =  [m.Var(0) for i in range(seg)] 
        y2 =  [m.Var(0) for i in range(seg)]
        y3 =  [m.Var(1) for i in range(seg)]
        v =  [m.Var(0) for i in range(seg)]
        q1 = [m.Var(0) for i in range(seg)]
        q2 = [m.Var(0) for i in range(seg)]
        P1 = m.Var(pressure_init)
        u = m.Var(Flow_rate*R*T/(Area*P1.value*10**2))
        P2 = m.Var(pressure_init)
    else:
        y1_feed = 0.55
        y2_feed = 0.45
        y3_feed = 0
        L_seg = L/seg
        Area = np.pi * D**2/4
        P = [m.Var(pressure_after_purge[-1][i]) for i in range(seg)] 
        y1 =  [m.Var(met_after_purge[-1][i]) for i in range(seg)] 
        y2 =  [m.Var(co2_after_purge[-1][i]) for i in range(seg)]
        y3 =  [m.Var(n2_after_purge[-1][i]) for i in range(seg)]
        v =  [m.Var(0) for i in range(seg)]
        q1 = [m.Var(q1_after_purge[-1][i]) for i in range(seg)]
        q2 = [m.Var(q2_after_purge[-1][i]) for i in range(seg)]
        P1 = m.Var(pressure_after_purge[-1][0])
        u = m.Var(Flow_rate*R*T/(Area*pressure_after_purge[-1][0]*10**2))
        P2 = m.Var(pressure_after_purge[-1][seg-1])

    m.Equation(u == Flow_rate*R*T/(Area*P1*10**2))
    m.Equation(P1.dt() ==  (P[0]/(V_tank))*(Flow_rate*10**(-2)*R*T/(P[0])-v[0]*Area))
    #Pressure Equation
    m.Equation(P2.dt() ==  (P2/(V_tank))*(v[seg-1]*Area))

    m.Equation(P[0]*y1[0].dt()+y1[0]*P[0].dt() == - (u/(et))*(y1[0]*(P[0]-P1)/(L_seg)+P[0]*(y1[0]-y1_feed)/(L_seg)) 
               - (P[0]*y1[0]/(et))*((v[0]-u)/(L_seg)) - (rho/(10**2*et))*R*T*q1[0].dt())

    m.Equation([P[i]*y1[i].dt()+y1[i]*P[i].dt() == - (v[i]/(et))*(y1[i]*(P[i]-P[i-1])/(L_seg)+P[i]*(y1[i]-y1[i-1])/(L_seg)) 
               - (P[i]*y1[i]/(et))*((v[i]-v[i-1])/(L_seg)) - (rho/(10**2*et))*R*T*q1[i].dt() for i in range(1,seg-1)])

    m.Equation(P[seg-1]*y1[seg-1].dt()+y1[seg-1]*P[seg-1].dt() == - (v[seg-1]/(et))*(y1[seg-1]*(P[seg-1]-P[seg-2])/(L_seg)
                                                                                     +P[seg-1]*(y1[seg-1]-y1[seg-2])/(L_seg)) 
               - (P[seg-1]*y1[seg-1]/(et))*((v[seg-1]-v[seg-2])/(L_seg)) - (rho/(10**2*et))*R*T*q1[seg-1].dt())


    m.Equation(P[0]*y2[0].dt()+y2[0]*P[0].dt() == - (u/(et))*(y2[0]*(P[0]-P1)/(L_seg)+P[0]*(y2[0]-y2_feed)/(L_seg)) 
               - (P[0]*y2[0]/(et))*((v[0]-u)/(L_seg)) - (rho/(10**2*et))*R*T*q2[0].dt())

    m.Equation([P[i]*y2[i].dt()+y2[i]*P[i].dt() == - (v[i]/(et))*(y2[i]*(P[i]-P[i-1])/(L_seg)+P[i]*(y2[i]-y2[i-1])/(L_seg)) 
               - (P[i]*y2[i]/(et))*((v[i]-v[i-1])/(L_seg)) - (rho/(10**2*et))*R*T*q2[i].dt() for i in range(1,seg-1)])

    m.Equation(P[seg-1]*y2[seg-1].dt()+y2[seg-1]*P[seg-1].dt() == - (v[seg-1]/(et))*(y2[seg-1]*(P[seg-1]-P[seg-2])/(L_seg)
                +P[seg-1]*(y2[seg-1]-y2[seg-2])/(L_seg)) 
               - (P[seg-1]**y2[seg-1]/(et))*((v[seg-1]-v[seg-2])/(L_seg)) - (rho/(10**2*et))*R*T*q2[seg-1].dt())

    m.Equation(P[0]*y3[0].dt()+y3[0]*P[0].dt() == - (u/(et))*(y3[0]*(P[0]-P1)/(L_seg)+P[0]*(y3[0]-y3_feed)/(L_seg)) 
               - (P[0]*y3[0]/(et))*((v[0]-u)/(L_seg)))

    m.Equation([P[i]*y3[i].dt()+y3[i]*P[i].dt() == - (v[i]/(et))*(y3[i]*(P[i]-P[i-1])/(L_seg)+P[i]*(y3[i]-y3[i-1])/(L_seg)) 
               - (P[i]*y3[i]/(et))*((v[i]-v[i-1])/(L_seg)) for i in range(1,seg-1)])

    m.Equation(P[seg-1]*y3[seg-1].dt()+y3[seg-1]*P[seg-1].dt() == - (v[seg-1]/(et))*(y3[seg-1]*(P[seg-1]-P[seg-2])/(L_seg)
                                                                                      +P[seg-1]*y3[seg-1]*(y3[seg-1]-y3[seg-2])/(L_seg)) 
               - (P[seg-1]*y3[seg-1]/(et))*((v[seg-1]-v[seg-2])/(L_seg)))


    m.Equation(v[0] == -(1/K)*(P[1]-P[0])/(L_seg))
    m.Equation([v[i] == -(1/K)*(P[i+1]-P[i])/(L_seg) for i in range(1,seg-1)])
    m.Equation(v[seg-1] == -(1/K)*(P2 - P[seg-1])/(L_seg))

    m.Equation([y1[i] +  y2[i]  == 1 - y3[i] for i in range(0,seg)])

    m.Equation([q1[i].dt() == -MTC[0]*(q1[i] - (IP1_CH4*y1[i]*P[i]*exp**(IP2_CH4/T)/(1+y1[i]*P[i]*IP3_CH4*exp**(IP4_CH4/T)))) for i in range(0,seg)])
    m.Equation([q2[i].dt() == -MTC[1]*(q2[i] - (IP1_CO2*y2[i]*P[i]*exp**(IP2_CO2/T)/(1+y2[i]*P[i]*IP3_CO2*exp**(IP4_CO2/T)))) for i in range(0,seg)])

    # simulation
    m.options.IMODE = 7
    m.solve(disp = False)
    print("FINIIIISHED Press")
    
    if cycle_count == number_cycles - 1:
        
        velocity_out = np.array(v[seg-1])
        pressure_out = np.array(P[seg-1])
        methane_out = np.array(y1[seg-1])
        co2_out = np.array(y2[seg-1])
        
        flow_methane =  velocity_out*Area*methane_out*pressure_out*10**8/(R*T)
        flow_co2 = velocity_out*Area*co2_out*pressure_out*10**8/(R*T)
        
        
        old_indices = np.arange(0,len(flow_methane))
        new_length = 100
        new_indices = np.linspace(0,len(flow_methane)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_methane,k=1,s=0)
        flow_methane  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(flow_co2))
        new_length = 100
        new_indices = np.linspace(0,len(flow_co2)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_co2,k=1,s=0)
        flow_co2  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(pressure_out))
        new_length = 100
        new_indices = np.linspace(0,len(pressure_out)-1,new_length)
        spl = UnivariateSpline(old_indices,pressure_out,k=1,s=0)
        pressure_out  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(m.time))
        new_length = 100
        new_indices = np.linspace(0,len(m.time)-1,new_length)
        spl = UnivariateSpline(old_indices,m.time,k=1,s=0)
        time  = np.abs(spl(new_indices))
        
        np_array_rows = time,pressure_out,flow_methane,flow_co2
        
        with open('results12.csv','a') as csvfile:
            np.savetxt(csvfile, np_array_rows , delimiter=',',header='Press',fmt='%s', comments='')
        
        
    met_after_press = np.empty((seg,len(m.time)))
    for i in range(seg):
        met_after_press[i] = np.array(y1[i].value)
    met_after_press= met_after_press.T

    co2_after_press = np.empty((seg,len(m.time)))
    for i in range(seg):
        co2_after_press[i] = np.array(y2[i].value)
    co2_after_press= co2_after_press.T


    n2_after_press = np.empty((seg,len(m.time)))
    for i in range(seg):
        n2_after_press[i] = np.array(y3[i].value)
    n2_after_press= n2_after_press.T

    q1_after_press = np.empty((seg,len(m.time)))
    for i in range(seg):
        q1_after_press[i] = np.array(q1[i].value)
    q1_after_press= q1_after_press.T

    q2_after_press = np.empty((seg,len(m.time)))
    for i in range(seg):
        q2_after_press[i] = np.array(q2[i].value)
    q2_after_press= q2_after_press.T
    
    pressure_after_adsorption = np.empty((seg,len(m.time)))
    for i in range(seg):
        pressure_after_adsorption[i] = np.array(P[i].value)
    pressure_after_adsorption = pressure_after_adsorption.T

    print("Staarted")
    MTC = [5.7E-5,0.025]
    m = GEKKO(remote = False)  # create GEKKO model
    tf =  100
    nt = int(tf/10) + 1
    m.time = np.linspace(0,tf,nt)
    y1_feed = 0.55
    y2_feed = 0.45
    y3_feed = 0
    L_seg = L/seg
    Area = np.pi * D**2/4
    P = [m.Var(pressure) for i in range(seg)] 
    y1 =  [m.Var(met_after_press[-1][i]) for i in range(seg)] 
    y2 =  [m.Var(co2_after_press[-1][i]) for i in range(seg)]
    y3 =  [m.Var(n2_after_press[-1][i]) for i in range(seg)]
    u_initial = Flow_rate*R*T/(Area*pressure*10**2)
    v =  [m.Var(0) for i in range(seg)]
    q1 = [m.Var(q1_after_press[-1][i]) for i in range(seg)]
    q2 = [m.Var(q2_after_press[-1][i]) for i in range(seg)]

    # first segment Pressure 1
    m.Equation(P[0]*y1[0].dt()+y1[0]*P[0].dt() == - (u_initial/(et))*(y1[0]*(P[0]-pressure)/(L_seg)+P[0]*(y1[0]-y1_feed)/(L_seg)) 
               - (P[0]*y1[0]/(et))*((v[0]-u_initial)/(L_seg)) - (rho/(10**2*et))*R*T*q1[0].dt())
    m.Equation([P[i]*y1[i].dt()+y1[i]*P[i].dt() == - (v[i]/(et))*(y1[i]*(P[i]-P[i-1])/(L_seg)+P[i]*(y1[i]-y1[i-1])/(L_seg)) 
               - (P[i]*y1[i]/(et))*((v[i]-v[i-1])/(L_seg)) - (rho/(10**2*et))*R*T*q1[i].dt() for i in range(1,seg-1)])
    m.Equation(P[seg-1]*y1[seg-1].dt()+y1[seg-1]*P[seg-1].dt() == - (v[seg-1]/(et))*(y1[seg-1]*(P[seg-1]-P[seg-2])/(L_seg)+P[seg-1]*(y1[seg-1]-y1[seg-2])/(L_seg)) 
               - (P[seg-1]*y1[seg-1]/(et))*((v[seg-1]-v[seg-2])/(L_seg)) - (rho/(10**2*et))*R*T*q1[seg-1].dt())


    m.Equation(P[0]*y2[0].dt()+y2[0]*P[0].dt() == - (u_initial/(et))*(y2[0]*(P[0]-pressure)/(L_seg)+P[0]*(y2[0]-y2_feed)/(L_seg)) 
               - (P[0]*y2[0]/(et))*((v[0]-u_initial)/(L_seg)) - (rho/(10**2*et))*R*T*q2[0].dt())

    m.Equation([P[i]*y2[i].dt()+y2[i]*P[i].dt() == - (v[i]/(et))*(y2[i]*(P[i]-P[i-1])/(L_seg)+P[i]*(y2[i]-y2[i-1])/(L_seg)) 
               - (P[i]*y2[i]/(et))*((v[i]-v[i-1])/(L_seg)) - (rho/(10**2*et))*R*T*q2[i].dt() for i in range(1,seg-1)])

    m.Equation(P[seg-1]*y2[seg-1].dt()+y2[seg-1]*P[seg-1].dt() == - (v[seg-1]/(et))*(y2[seg-1]*(P[seg-1]-P[seg-2])/(L_seg)
                +P[seg-1]*(y2[seg-1]-y2[seg-2])/(L_seg)) 
               - (P[seg-1]**y2[seg-1]/(et))*((v[seg-1]-v[seg-2])/(L_seg)) - (rho/(10**2*et))*R*T*q2[seg-1].dt())

    m.Equation(P[0]*y3[0].dt()+y3[0]*P[0].dt() == - (u_initial/(et))*(y3[0]*(P[0]-pressure)/(L_seg)+P[0]*(y3[0]-y3_feed)/(L_seg)) 
               - (P[0]*y3[0]/(et))*((v[0]-u_initial)/(L_seg)))
    m.Equation([P[i]*y3[i].dt()+y3[i]*P[i].dt() == - (v[i]/(et))*(y3[i]*(P[i]-P[i-1])/(L_seg)+P[i]*(y3[i]-y3[i-1])/(L_seg)) 
               - (P[i]*y3[i]/(et))*((v[i]-v[i-1])/(L_seg)) for i in range(1,seg-1)])
    m.Equation(P[seg-1]*y3[seg-1].dt()+y3[seg-1]*P[seg-1].dt() == - (v[seg-1]/(et))*(y3[seg-1]*(P[seg-1]-P[seg-2])/(L_seg)
                                                                                      +P[seg-1]*y3[seg-1]*(y3[seg-1]-y3[seg-2])/(L_seg)) 
               - (P[seg-1]*y3[seg-1]/(et))*((v[seg-1]-v[seg-2])/(L_seg)))

    m.Equation(v[0] == -(1/K)*(P[0]-pressure)/(L_seg))
    m.Equation([v[i] == -(1/K)*(P[i]-P[i-1])/(L_seg) for i in range(1,seg-1)])
    m.Equation(v[seg-1] == -(1/K)*(P[seg-1]-P[seg-2])/(L_seg))

    m.Equation([y1[i] +  y2[i]  == 1 - y3[i] for i in range(0,seg)])

    m.Equation([q1[i].dt() == -MTC[0]*(q1[i] - (IP1_CH4*y1[i]*P[i]*exp**(IP2_CH4/T)/(1+y1[i]*P[i]*IP3_CH4*exp**(IP4_CH4/T)))) for i in range(0,seg)])
    m.Equation([q2[i].dt() == -MTC[1]*(q2[i] - (IP1_CO2*y2[i]*P[i]*exp**(IP2_CO2/T)/(1+y2[i]*P[i]*IP3_CO2*exp**(IP4_CO2/T)))) for i in range(0,seg)])

    # simulation
    m.options.IMODE = 7
    m.solve(disp = False)
    print("FINIIIISHED ads")

    met_after_ads= np.empty((seg,len(m.time)))
    for i in range(seg):
        met_after_ads[i] = np.array(y1[i].value)
    met_after_ads= met_after_ads.T

    co2_after_ads = np.empty((seg,len(m.time)))
    for i in range(seg):
        co2_after_ads[i] = np.array(y2[i].value)
    co2_after_ads= co2_after_ads.T


    n2_after_ads = np.empty((seg,len(m.time)))
    for i in range(seg):
        n2_after_ads[i] = np.array(y3[i].value)
    n2_after_ads= n2_after_ads.T

    q1_after_ads = np.empty((seg,len(m.time)))
    for i in range(seg):
        q1_after_ads[i] = np.array(q1[i].value)
    q1_after_ads= q1_after_ads.T

    q2_after_ads = np.empty((seg,len(m.time)))
    for i in range(seg):
        q2_after_ads[i] = np.array(q2[i].value)
    q2_after_ads= q2_after_ads.T
    if cycle_count == number_cycles - 1:
        velocity_out = np.array(v[seg-1])
        pressure_out = np.array(P[seg-1])
        methane_out = np.array(y1[seg-1])
        co2_out = np.array(y2[seg-1])
        flow_methane =  velocity_out*Area*methane_out*pressure_out*10**8/(R*T)
        flow_co2 = velocity_out*Area*co2_out*pressure_out*10**8/(R*T)
        
        
        old_indices = np.arange(0,len(flow_methane))
        new_length = 100
        new_indices = np.linspace(0,len(flow_methane)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_methane,k=1,s=0)
        flow_methane  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(flow_co2))
        new_length = 100
        new_indices = np.linspace(0,len(flow_co2)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_co2,k=1,s=0)
        flow_co2  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(pressure_out))
        new_length = 100
        new_indices = np.linspace(0,len(pressure_out)-1,new_length)
        spl = UnivariateSpline(old_indices,pressure_out,k=1,s=0)
        pressure_out  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(m.time))
        new_length = 100
        new_indices = np.linspace(0,len(m.time)-1,new_length)
        spl = UnivariateSpline(old_indices,m.time,k=1,s=0)
        time  = np.abs(spl(new_indices))
        
        np_array_rows = time + 70,pressure_out,flow_methane,flow_co2
        
        with open('results12.csv','a') as csvfile:
            np.savetxt(csvfile, np_array_rows , delimiter=',',header='ADS',fmt='%s', comments='')

    print("Staarted")
    MTC = [5.7E-5,0.025]
    m = GEKKO(remote = False)  # create GEKKO model
    tf = 100
    nt = int(tf/1) + 1
    m.time = np.linspace(0,tf,nt)

    L_seg = L/seg
    Area = np.pi * D**2/4
    P = [m.Var(pressure) for i in range(seg)] 
    y1 =  [m.Var(met_after_ads[-1][i]) for i in range(seg)] 
    y2 =  [m.Var(co2_after_ads[-1][i]) for i in range(seg)]
    y3 =  [m.Var(n2_after_ads[-1][i]) for i in range(seg)]
    v =  [m.Var(0) for i in range(seg)]
    q1 = [m.Var(q1_after_ads[-1][i]) for i in range(seg)]
    q2 = [m.Var(q2_after_ads[-1][i]) for i in range(seg)]
    P1 = m.Var(pressure)
    P2 = m.Var(pressure)

    Q_pump   =  m.Var(0.8 / 3600)
    m.Equation(P2.dt() ==  (P2/(V_tank))*(v[seg-1]*Area))
    m.Equation(P1.dt() == (-((P[0])*Q_pump)/(V_tank) - (P[0]*v[0]*Area)/(V_tank)))
    m.Equation(Q_pump ==  0.0001974563*P[0]**(0.627027))

    # first segment Pressure 1
    m.Equation(P[0]*y1[0].dt()+y1[0]*P[0].dt() == (1/(K*et))*((y1[0]*P[0]*(P[1] - 2*P[0] + P1)/(L_seg**2))
                                                          + P[1]*((P[1]-P[0])/(L_seg))*((y1[1]-y1[0])/(L_seg))
                                                          + y1[1]*((P[1]-P[0])/(L_seg))**2) 
    - ((1/(et*100))*(R*T)*rho*q1[0].dt()))

    # middle segments Pressure 1
    m.Equations([P[i]*y1[i].dt()+y1[i]*P[i].dt() ==  \
                (1/(K*et))*((y1[i]*P[i]*(P[i+1]-2*P[i]+P[i-1])/(L_seg**2)) + P[i+1]*((P[i+1]-P[i])/(L_seg))*((y1[i+1]-y1[i])/(L_seg)) 
                + y1[i+1]*((P[i+1]-P[i])/(L_seg))**2) - (1/(et*100))*(R*T)*rho*q1[i].dt()   for i in range(1,seg-1)])

    # last segment Pressure 1
    m.Equation(P[seg-1]*y1[seg-1].dt()+y1[seg-1]*P[seg-1].dt() == \
            (1/(K*et))*((y1[seg-1]*P[seg-1]*(-P[seg-1]+P[seg-2])/(L_seg**2))) - ((1/(et*100))*(R*T)*rho*q1[seg-1].dt()))

    # first segment Pressure 2
    m.Equation(P[0]*y2[0].dt()+y2[0]*P[0].dt() == (1/(K*et))*((y2[0]*P[0]*(P[1] - 2*P[0] + P1)/(L_seg**2)) + P[1]*((P[1]-P[0])/(L_seg))*((y2[1]-y2[0])/(L_seg)) 
    + y2[1]*((P[1]-P[0])/(L_seg))**2) - ((1/(et*100))*(R*T)*rho*q2[0].dt())) 

    # middle segments Pressure 2
    m.Equations([P[i]*y2[i].dt()+y2[i]*P[i].dt() ==  \
                (1/(K*et))*((y2[i]*P[i]*(P[i+1]-2*P[i]+P[i-1])/(L_seg**2)) +  P[i+1]*((P[i+1]-P[i])/(L_seg))*((y2[i+1]-y2[i])/(L_seg)) 
                + y2[i+1]*((P[i+1]-P[i])/(L_seg))**2) - ((1/(et*100))*(R*T))*rho*q2[i].dt()   for i in range(1,seg-1)])

    # last segment Pressure 
    m.Equation(P[seg-1]*y2[seg-1].dt()+y2[seg-1]*P[seg-1].dt() == \
            (1/(K*et))*((y2[seg-1]*P[seg-1]*(-P[seg-1]+P[seg-2])/(L_seg**2))) - ((1/(et*100))*(R*T))*rho*q2[seg-1].dt())



    # first segment Pressure 3
    m.Equation(P[0]*y3[0].dt()+y3[0]*P[0].dt() == (1/(K*et))*((y3[0]*P[0]*(P[1] - 2*P[0] + P1)/(L_seg**2)) 
    + P[1]*((P[1]-P[0])/(L_seg))*((y3[1]-y3[0])/(L_seg)) + y3[1]*((P[1]-P[0])/(L_seg))**2))

    # middle segments Pressure 3
    m.Equations([P[i]*y3[i].dt()+y3[i]*P[i].dt() ==  \
                (1/(K*et))*((y3[i]*P[i]*(P[i+1]-2*P[i]+P[i-1])/(L_seg**2)) + P[i+1]*((P[i+1]-P[i])/(L_seg))*((y3[i+1]-y3[i])/(L_seg)) 
                + y3[i+1]*((P[i+1]-P[i])/(L_seg))**2)  for i in range(1,seg-1)])

    # last segment Pressure 3
    m.Equation(P[seg-1]*y3[seg-1].dt()+y3[seg-1]*P[seg-1].dt() == \
            (1/(K*et))*((y3[seg-1]*P[seg-1]*(-P[seg-1]+P[seg-2])/(L_seg**2))))

    m.Equation(v[0] == -(1/K)*(P[1]-P[0])/(L_seg))
    m.Equation([v[i] == -(1/K)*(P[i+1]-P[i])/(L_seg) for i in range(1,seg-1)])
    m.Equation(v[seg-1] == -(1/K)*(P2 - P[seg-1])/(L_seg))

    m.Equation([y1[i] +  y2[i]  == 1 - y3[i] for i in range(0,seg)])

    m.Equation([q1[i].dt() == -MTC[0]*(q1[i] - (IP1_CH4*y1[i]*P[i]*exp**(IP2_CH4/T)/(1+y1[i]*P[i]*IP3_CH4*exp**(IP4_CH4/T)))) for i in range(0,seg)])
    m.Equation([q2[i].dt() == -MTC[1]*(q2[i] - (IP1_CO2*y2[i]*P[i]*exp**(IP2_CO2/T)/(1+y2[i]*P[i]*IP3_CO2*exp**(IP4_CO2/T)))) for i in range(0,seg)])


    # simulation
    m.options.IMODE = 7
    m.solve(disp=False)
    print("Finished DESORP")
    
            
    met_after_desorp= np.empty((seg,len(m.time)))
    for i in range(seg):
        met_after_desorp[i] = np.array(y1[i].value)
    met_after_desorp= met_after_desorp.T

    co2_after_desorp= np.empty((seg,len(m.time)))
    for i in range(seg):
        co2_after_desorp[i] = np.array(y2[i].value)
    co2_after_desorp= co2_after_desorp.T


    n2_after_desorp = np.empty((seg,len(m.time)))
    for i in range(seg):
        n2_after_desorp[i] = np.array(y3[i].value)
    n2_after_desorp= n2_after_desorp.T

    q1_after_desorp = np.empty((seg,len(m.time)))
    for i in range(seg):
        q1_after_desorp[i] = np.array(q1[i].value)
    q1_after_desorp= q1_after_desorp.T

    q2_after_desorp = np.empty((seg,len(m.time)))
    for i in range(seg):
        q2_after_desorp[i] = np.array(q2[i].value)
    q2_after_desorp= q2_after_desorp.T

    v_after_desorp = np.empty((seg,len(m.time)))
    for i in range(seg):
        v_after_desorp[i] = np.array(v[i].value)
    v_after_desorp= v_after_desorp.T

    pressure_after_desorp = np.empty((seg,len(m.time)))
    for i in range(seg):
        pressure_after_desorp[i] = np.array(P[i].value)
    pressure_after_desorp= pressure_after_desorp.T

    v_after_desorp = np.empty((seg,len(m.time)))
    for i in range(seg):
        v_after_desorp[i] = np.array(v[i].value)
    v_after_desorp= v_after_desorp.T
    
    if cycle_count == number_cycles - 1:
        velocity_out = -1*np.array(v[0])
        pressure_out = np.array(P[0])
        methane_out = np.array(y1[0])
        co2_out = np.array(y2[0])
        flow_methane =  velocity_out*Area*methane_out*pressure_out*10**8/(R*T)
        flow_co2 = velocity_out*Area*co2_out*pressure_out*10**8/(R*T)

        
        old_indices = np.arange(0,len(flow_methane))
        new_length = 100
        new_indices = np.linspace(0,len(flow_methane)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_methane,k=1,s=0)
        flow_methane  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(flow_co2))
        new_length = 100
        new_indices = np.linspace(0,len(flow_co2)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_co2,k=1,s=0)
        flow_co2  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(pressure_out))
        new_length = 100
        new_indices = np.linspace(0,len(pressure_out)-1,new_length)
        spl = UnivariateSpline(old_indices,pressure_out,k=1,s=0)
        pressure_out  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(m.time))
        new_length = 100
        new_indices = np.linspace(0,len(m.time)-1,new_length)
        spl = UnivariateSpline(old_indices,m.time,k=1,s=0)
        time  = np.abs(spl(new_indices))
        
        np_array_rows = time+170,pressure_out,flow_methane,flow_co2
        
        with open('results12.csv','a') as csvfile:
            np.savetxt(csvfile, np_array_rows , delimiter=',',header='des',fmt='%s', comments='')

    print("Staarted")
    MTC = [5.7E-5,0.025]
    m = GEKKO(remote = False)  # create GEKKO model
    tf = 70
    nt = int(tf/1) + 1
    m.time = np.linspace(0,tf,nt)

    y1_feed = 1
    y2_feed = 0
    y3_feed = 0

    L_seg = L/seg
    Area = np.pi * D**2/4
    P = [m.Var(pressure_after_desorp[-1][i]) for i in range(seg)] 
    y1 =  [m.Var(met_after_desorp[-1][i]) for i in range(seg)] 
    y2 =  [m.Var(co2_after_desorp[-1][i]) for i in range(seg)]
    y3 =  [m.Var(n2_after_desorp[-1][i]) for i in range(seg)]
    v =  [m.Var(v_after_desorp[-1][i]) for i in range(seg)]
    q1 = [m.Var(q1_after_desorp[-1][i]) for i in range(seg)]
    q2 = [m.Var(q2_after_desorp[-1][i]) for i in range(seg)]
    P1 = m.Var(pressure_after_desorp[-1][0])
    P2 = m.Var(pressure_after_desorp[-1][-1])

    Q_pump   =  m.Var(0.6 / 3600)

    m.Equation(Q_pump ==  0.0001974563*P[0]**(0.627027))

    m.Equation(P1.dt() == (-((P[0])*Q_pump)/(V_tank) - (P[0]*v[0]*Area)/(V_tank)))

    m.Equation(P2.dt() ==  (P[seg-1]/(V_tank))*(Flow_rate_purge*10**(-2)*R*T/(P[seg-1])+v[seg-1]*Area))


    #Pressure Equation
    # first segment Pressure 1
    m.Equation(P[0]*y1[0].dt()+y1[0]*P[0].dt() == (1/(K*et))*((y1[0]*P[0]*(P[1]-2*P[0]+P1)/(L_seg**2))
                                                          + (P[1])*((P[1]- P[0])/(L_seg))*((y1[1]-y1[0])/(L_seg)) 
    + (y1[1])*((P[1]-P[0])/(L_seg))**2) - ((1/(et*100))*(R*T))*rho*q1[0].dt())

    # middle segments Pressure 1
    m.Equations([P[i]*y1[i].dt()+y1[i]*P[i].dt() ==  \
                (1/(K*et))*((y1[i]*P[i]*(P[i+1]-2*P[i]+P[i-1])/(L_seg**2)) + (P[i+1])*((P[i+1]-P[i])/(L_seg))*((y1[i+1]-y1[i])/(L_seg)) 
                + (y1[i+1])*((P[i+1]-P[i])/(L_seg))**2) - ((1/(et*100))*(R*T))*rho*q1[i].dt()  for i in range(1,seg-1)])

    # last segment Pressure 1
    m.Equation(P[seg-1]*y1[seg-1].dt()+y1[seg-1]*P[seg-1].dt() == \
            (1/(K*et))*((y1[seg-1]*P[seg-1]*(P2-2*P[seg-1]+P[seg-2])/(L_seg**2)) 
            + (P2)*((P2-P[seg-1])/(L_seg))*((y1_feed-y1[seg-1])/(L_seg))
                    + (y1_feed)*((P2-P[seg-1])/(L_seg))**2) - ((1/(et*100))*(R*T))*rho*q1[seg-1].dt())

    # first segment Pressure 2
    m.Equation(P[0]*y2[0].dt()+y2[0]*P[0].dt() == (1/(K*et))*((y2[0]*P[0]*(P[1]-2*P[0]+P1)/(L_seg**2)) 
    + (P[1])*((P[1]- P[0])/(L_seg))*((y2[1]-y2[0])/(L_seg))
                                                          + (y2[1])*((P[1]-P[0])/(L_seg))**2) 
               - ((1/(et*100))*(R*T))*rho*q2[0].dt())

    # middle segments Pressure 2
    m.Equations([P[i]*y2[i].dt()+y2[i]*P[i].dt() ==  \
                (1/(K*et))*((y2[i]*P[i]*(P[i+1]-2*P[i]+P[i-1])/(L_seg**2)) 
                + (P[i+1])*((P[i+1]-P[i])/(L_seg))*((y2[i+1]-y2[i])/(L_seg)) + (y2[i+1])*((P[i+1]-P[i])/(L_seg))**2) 
                - ((1/(et*100))*(R*T))*rho*q2[i].dt()   for i in range(1,seg-1)])

    # last segment Pressure 2
    m.Equation(P[seg-1]*y2[seg-1].dt()+y2[seg-1]*P[seg-1].dt() == \
            (1/(K*et))*((y2[seg-1]*P[seg-1]*(P2-2*P[seg-1]+P[seg-2])/(L_seg**2)) 
            + (P2)*((P2-P[seg-1])/(L_seg))*((y2_feed-y2[seg-1])/(L_seg)) 
                    + (y2_feed)*((P2-P[seg-1])/(L_seg))**2) - ((1/(et*100))*(R*T))*rho*q2[seg-1].dt())

    # first segment Pressure 3
    m.Equation(P[0]*y3[0].dt()+y3[0]*P[0].dt() == (1/(K*et))*((y3[0]*P[0]*(P[1]-2*P[0]+P1)/(L_seg**2)) 
    + (P[1])*((P[1]- P[0])/(L_seg))*((y3[1]-y3[0])/(L_seg)) 
                                                          + (y3[1])*((P[1]-P[0])/(L_seg))**2) 
               )

    # middle segments Pressure 3
    m.Equations([P[i]*y3[i].dt()+y3[i]*P[i].dt() ==  \
                (1/(K*et))*((y3[i]*P[i]*(P[i+1]-2*P[i]+P[i-1])/(L_seg**2)) 
                        + (P[i+1])*((P[i+1]-P[i])/(L_seg))*((y3[i+1]-y3[i])/(L_seg)) 
                        + (y3[i+1])*((P[i+1]-P[i])/(L_seg))**2) 
                 for i in range(1,seg-1)])

    # last segment Pressure 3
    m.Equation(P[seg-1]*y3[seg-1].dt()+y3[seg-1]*P[seg-1].dt() == 
            (1/(K*et))*((y3[seg-1]*P[seg-1]*(P2-2*P[seg-1]+P[seg-2])/(L_seg**2))
                    + (P2)*((P2-P[seg-1])/(L_seg))*((y3_feed-y3[seg-1])/(L_seg)) + (y3_feed)*((P2-P[seg-1])/(L_seg))**2) 
            )


    m.Equation(v[0] == -(1/K)*(P[1]-P[0])/(L_seg))
    m.Equation([v[i] == -(1/K)*(P[i+1]-P[i])/(L_seg) for i in range(1,seg-1)])
    m.Equation(v[seg-1] == -(1/K)*(P2 - P[seg-1])/(L_seg))

    m.Equation([y1[i] +  y2[i]  == 1 - y3[i] for i in range(0,seg)])

    m.Equation([q1[i].dt() == -MTC[0]*(q1[i] - (IP1_CH4*y1[i]*P[i]*exp**(IP2_CH4/T)/(1+y1[i]*P[i]*IP3_CH4*exp**(IP4_CH4/T)))) for i in range(0,seg)])
    m.Equation([q2[i].dt() == -MTC[1]*(q2[i] - (IP1_CO2*y2[i]*P[i]*exp**(IP2_CO2/T)/(1+y2[i]*P[i]*IP3_CO2*exp**(IP4_CO2/T)))) for i in range(0,seg)])


    # simulation
    m.options.IMODE = 7
    m.solve(disp = False)

    print("Finished PURGE")
    
    if cycle_count == number_cycles - 1:
        velocity_out = -1*np.array(v[0])
        pressure_out = np.array(P[0])
        methane_out = np.array(y1[0])
        co2_out = np.array(y2[0])
        flow_methane =  velocity_out*Area*methane_out*pressure_out*10**8/(R*T)
        flow_co2 = velocity_out*Area*co2_out*pressure_out*10**8/(R*T)
        
        
        old_indices = np.arange(0,len(flow_methane))
        new_length = 100
        new_indices = np.linspace(0,len(flow_methane)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_methane,k=1,s=0)
        flow_methane  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(flow_co2))
        new_length = 100
        new_indices = np.linspace(0,len(flow_co2)-1,new_length)
        spl = UnivariateSpline(old_indices,flow_co2,k=1,s=0)
        flow_co2  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(pressure_out))
        new_length = 100
        new_indices = np.linspace(0,len(pressure_out)-1,new_length)
        spl = UnivariateSpline(old_indices,pressure_out,k=1,s=0)
        pressure_out  = np.abs(spl(new_indices))
        
        old_indices = np.arange(0,len(m.time))
        new_length = 100
        new_indices = np.linspace(0,len(m.time)-1,new_length)
        spl = UnivariateSpline(old_indices,m.time,k=1,s=0)
        time  = np.abs(spl(new_indices))
        
        np_array_rows = time+270,pressure_out,flow_methane,flow_co2
        
        with open('results12.csv','a') as csvfile:
            np.savetxt(csvfile, np_array_rows , delimiter=',',header='P',fmt='%s', comments='')
            
    met_after_purge= np.empty((seg,len(m.time)))
    for i in range(seg):
        met_after_purge[i] = np.array(y1[i].value)
    met_after_purge= met_after_purge.T

    co2_after_purge= np.empty((seg,len(m.time)))
    for i in range(seg):
        co2_after_purge[i] = np.array(y2[i].value)
    co2_after_purge= co2_after_purge.T


    n2_after_purge = np.empty((seg,len(m.time)))
    for i in range(seg):
        n2_after_purge[i] = np.array(y3[i].value)
    n2_after_purge= n2_after_purge.T

    q1_after_purge = np.empty((seg,len(m.time)))
    for i in range(seg):
        q1_after_purge[i] = np.array(q1[i].value)
    q1_after_purge= q1_after_purge.T

    q2_after_purge = np.empty((seg,len(m.time)))
    for i in range(seg):
        q2_after_purge[i] = np.array(q2[i].value)
    q2_after_purge= q2_after_purge.T

    pressure_after_purge = np.empty((seg,len(m.time)))
    for i in range(seg):
        pressure_after_purge[i] = np.array(P[i].value)
    pressure_after_purge= pressure_after_purge.T


Staarted cycle 1
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 2
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 3
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 4
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 5
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 6
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 7
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 8
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 9
FINIIIISHED Press
Staarted
FINIIIISHED ads
Staarted
Finished DESORP
Staarted
Finished PURGE
Staarted cycle 10
F